In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('data-dransformer') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/07 21:31:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
pyspark.__version__

'3.2.1'